# Laboratorium 3: Przetwarzanie i Analiza Danych w Pandas

**Cel ćwiczenia:** Celem tego laboratorium jest przećwiczenie zaawansowanych technik czyszczenia i transformacji danych z wykorzystaniem biblioteki Pandas. Będziemy pracować na przykładowym zbiorze danych, symulującym odczyty z sensorów przemysłowych.

---

### **Zadanie 1: Wczytanie i porządkowanie danych**

Załaduj dane do obiektu DataFrame, posortuj je chronologicznie według kolumny `timestamp`, a następnie ustaw tę kolumnę jako nowy indeks. Na koniec usuń zbędną kolumnę `index`.

> **Wskazówka:** Użyj `pd.read_csv()` do załadowania danych oraz `sort_values()`,`set_index()` dla ustawienia indeksu na `timestamp`.

In [8]:
# Krok 1: Zaimportuj bibliotekę pandas
import pandas as pd

# Krok 2: Wczytaj dane z pliku CSV
df = pd.read_csv('technician_measurements_with_errors.csv')

df_sorted = df.sort_values(by='timestamp')

df_indexed=df_sorted.set_index('timestamp')

df_final=df_indexed.drop(columns=['index'])
# Wyświetl pierwsze 5 wierszy, aby zweryfikować zmiany
df.head()

,index,timestamp,voltage,current,temperature,technician_first_name,technician_last_name,status
0,1,2023-01-09 18:59:00,999.000000,1.873448,54.312392,John,Doe,working
1,2,2023-01-17 05:49:00,0.000000,0.000000,64.919201,Jane,Smith,turned off
2,3,2023-01-23 14:33:00,226.006387,1.932824,58.203752,Emily,Brown,working
3,4,2023-01-24 05:55:00,227.497793,1.455068,69.815826,Michael,Johnson,working
4,5,2023-01-25 09:08:00,0.000000,0.000000,56.077541,John,Doe,turned off


---

### **Zadanie 2: Łączenie kolumn**

Połącz dane z kolumn `technician_first_name` i `technician_last_name` w jedną nową kolumnę o nazwie `technician`. Dane rozdziel spacją.

In [1]:
df['technician_first_name']+' '+df['technician_last_name']

NameError: name 'df' is not defined

---

### **Zadanie 3: Kodowanie zero-jedynkowe (One-Hot Encoding)**

Przekształć kolumnę kategoryczną `status` na trzy nowe kolumny binarne: `status_working`, `status_broken` i `status_turned_off`. Każda z nowych kolumn powinna przyjmować wartość `1`, jeśli status w oryginalnej kolumnie odpowiada jej nazwie, a `0` w przeciwnym razie.

> **Wskazówka:** Skorzystaj z funkcji `pd.get_dummies()`.

In [16]:
status_dummies = pd.get_dummies(df['status'], prefix='status')
df_encoded = pd.concat([df, status_dummies], axis =1 )
df_encoded = df_encoded.drop(columns=
    ['status'])
df_encoded.head()

,index,timestamp,voltage,current,temperature,technician_first_name,technician_last_name,status_broken,status_turned off,status_working
0,1,2023-01-09 18:59:00,999.000000,1.873448,54.312392,John,Doe,False,False,True
1,2,2023-01-17 05:49:00,0.000000,0.000000,64.919201,Jane,Smith,False,True,False
2,3,2023-01-23 14:33:00,226.006387,1.932824,58.203752,Emily,Brown,False,False,True
3,4,2023-01-24 05:55:00,227.497793,1.455068,69.815826,Michael,Johnson,False,False,True
4,5,2023-01-25 09:08:00,0.000000,0.000000,56.077541,John,Doe,False,True,False


---

### **Zadanie 4: Usuwanie wartości odstających (outliers)**

Zidentyfikuj i usuń wiersze zawierające *wartości odstające* w kolumnach `current` i `voltage`. Za wartość odstającą uznajemy każdą wartość, która jest **większa niż 95. [percentyl](https://www.wikiwand.com/pl/articles/Percentyl)** dla danej kolumny.

> **Wskazówka:** Użyj metody `.quantile()`

---

### **Zadanie 5: Zastępowanie wartości odstających**

Tym razem, zamiast usuwać, zastąp wartości odstające w kolumnie `temperature`. Zastąp je **średnią temperaturą**, obliczoną osobno dla każdej kategorii ze starej kolumny `status` (`'broken'`, `'working'`, itd.).

> **Wskazówka:** Możesz użyć `groupby()` i `transform()`, aby obliczyć średnie dla grup.

---
### **Zadanie 6: Inżynieria cech (Feature Engineering)**

Dodaj dwie nowe kolumny:
1.  `power` – obliczoną jako iloczyn wartości z kolumn `voltage` i `current`.
2.  `normalized_power` – zawierającą znormalizowane wartości z kolumny `power`.

Do normalizacji użyj poniższego wzoru (skalowanie min-max):
$$ \text{Normalized} = \frac{\text{Value} - \text{Min}}{\text{Max} - \text{Min}} $$

---

### **Zadanie 7: Standaryzacja danych**

Dokonaj standaryzacji wartości w kolumnie `temperature`. Standaryzacja (lub normalizacja Z-score) polega na przekształceniu danych tak, aby ich średnia była równa `0`, a odchylenie standardowe równe `1`.
$$ \text{Standardized} = \frac{\text{Value} - \text{Mean}}{\text{Std}} $$

> **Wskazówka:** Użyj metody `.apply()` wraz z funkcją `lambda`.


---

### **Zadanie 8: Modyfikacja nazw kolumn**

Zmień nazwy wszystkich kolumn tak, aby każda z nich zaczynała się wielką literą (np. `voltage` -> `Voltage`).

> **Wskazówka:** Wykorzystaj metodę `.rename()` w połączeniu z funkcją `str.title()`.

---

### **Zadanie 9: Warunkowa modyfikacja danych**

Do każdego rekordu w kolumnie `Current`, który został zarejestrowany **przed datą 10 stycznia**, dodaj wartość `0.1`.

> **Wskazówka:** Użyj `loc[]` do warunkowego wybrania i zaktualizowania odpowiednich wierszy.